# 2. (Deep) CNNs for Image Colorization
Ruotian Jiang 8389636738

# a & b 
The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

In [22]:
import numpy as np
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict
# use extract to get bird class
def extract(dict):
    index = [i for i,x in enumerate(dict['labels']) if x == 2]
    return dict['data'][index]
# change shape to that Each pixel is an RGB vector with three elements.
def reshape(array):
    result = []
    for i in array:
        a = np.reshape(i, (-1, 1024))
        result.append(a.T)
    return np.array(result)
data1 = reshape(extract(unpickle('data_batch_1')))
data2 = reshape(extract(unpickle('data_batch_2')))
data3 = reshape(extract(unpickle('data_batch_3')))
data4 = reshape(extract(unpickle('data_batch_4')))
data5 = reshape(extract(unpickle('data_batch_5')))
data6 = reshape(extract(unpickle('test_batch')))

In [28]:
print 'test size'
print data6.shape

test size
(1000, 1024, 3)


# c

In [ ]:
# here I Choose 50% of the pixels randomly.

# d

In [26]:
from sklearn.cluster import KMeans
X = np.concatenate((data1, data2, data3, data4, data5, data6))
#  transform to P vectors
X = X.reshape(6144000,3)
kmeans = KMeans(n_clusters=4).fit(X)

In [57]:
trans = dict({0: kmeans.cluster_centers_[0],1: kmeans.cluster_centers_[1],2: kmeans.cluster_centers_[2],3: kmeans.cluster_centers_[3]})
print trans

{0: array([156.31445571, 155.64326769, 135.724557  ]), 1: array([49.07644711, 50.05637534, 38.59833484]), 2: array([206.62474607, 211.87845177, 211.53547018]), 3: array([109.05965754, 108.70112922,  82.76727714])}


In [133]:
# y = np.array([trans[i] for i in kmeans.labels_])

y = np.array(kmeans.labels_)




In [134]:
y = y.reshape(6000,32,32,1)
y_train = y[:5000]
y_test = y[:1000]
print y_test.shape



(1000, 32, 32, 1)


# e

In [42]:
from skimage.color import rgb2gray
X = np.concatenate((data1, data2, data3, data4, data5, data6))
print X[0]
X = X.reshape(6000,32,32,3)
print X[0]
X = rgb2gray(X)
X = X.reshape(6000,32,32,1)
X_train = X[:5000]
X_test = X[:1000]
print X_train.shape

[[164 206  84]
 [105 140  61]
 [118 148 101]
 ...
 [ 94 134  29]
 [ 91 133  26]
 [122 170  44]]
[[[164 206  84]
  [105 140  61]
  [118 148 101]
  ...
  [109 147  73]
  [108 147  69]
  [ 91 129  57]]

 [[167 213  84]
  [116 160  49]
  [ 72 109  43]
  ...
  [105 142  79]
  [105 142  72]
  [ 89 127  57]]

 [[140 191  65]
  [142 193  66]
  [119 163  79]
  ...
  [104 139  84]
  [ 84 120  58]
  [ 78 115  49]]

 ...

 [[139 148  81]
  [142 155  74]
  [135 156  72]
  ...
  [ 89 134  28]
  [ 97 148  24]
  [126 176  49]]

 [[163 157  85]
  [153 164  82]
  [146 164  90]
  ...
  [ 85 130  19]
  [ 98 148  27]
  [127 178  48]]

 [[183 153 102]
  [176 182 116]
  [154 154 100]
  ...
  [ 94 134  29]
  [ 91 133  26]
  [122 170  44]]]
(5000, 32, 32, 1)


# f
here I used test data for validation



In [148]:
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np

#Adding Seed so that random initialization is consistent
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


batch_size = 100

#Prepare input data
num_classes = 4



print("Complete reading input data. Will Now print a snippet of it")
print("Number of files in Training-set:\t\t{}".format(len(X_train)))
print("Number of files in Validation(test)-set:\t{}".format(len(X_test)))



session = tf.Session()

## labels
num_channels = 1
x = tf.placeholder(tf.float32, shape=[None, 32,32,1], name='x')
 
y_true = tf.placeholder(tf.float32, shape=[None,1], name='y_true')
# y_true = tf.reshape(y_true, [-1,3])
y_true_cls = tf.argmax(y_true, dimension=1)


##Network graph params
filter_size_conv1 = 5 
num_filters_conv1 = 16

filter_size_conv2 = 5
num_filters_conv2 = 16
   


def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))



def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    layer = tf.nn.relu(layer)

    return layer

def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    layer = tf.reshape(layer, [-1, 1])
    
    return layer

def create_mlp_layer(input,  
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer 


layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_flat = create_flatten_layer(layer_conv2)

layer_mlp1 = create_mlp_layer(input=layer_flat,
                     num_inputs=1,
                     num_outputs=32,
                     use_relu=True)

layer_mlp2 = create_mlp_layer(input=layer_mlp1,
                     num_inputs=32,
                     num_outputs=1,
                     use_relu=True)

y_pred = tf.nn.softmax(layer_mlp2,name='y_pred')
y_pred_cls = tf.argmax(y_pred, dimension=1)



session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_mlp2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 


def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation(test) Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))



saver = tf.train.Saver()
def train(num_iteration):
    
    for i in range(num_iteration):
        for j in range(50):
            x_batch = X_train[batch_size*j:batch_size*(j+1)]
            y_true_batch = y_train[batch_size*j:batch_size*(j+1)]
            y_true_batch = y_true_batch.reshape(-1,1)

            x_valid_batch = X_test[batch_size*(j%5):batch_size*((j%5)+1)]
            y_valid_batch = y_test[batch_size*(j%5):batch_size*((j%5)+1)]
            y_valid_batch = y_valid_batch.reshape(-1,1)

            feed_dict_tr = {x: x_batch,
                               y_true: y_true_batch}
            feed_dict_val = {x: x_valid_batch,
                                  y_true: y_valid_batch}
            session.run(optimizer, feed_dict=feed_dict_tr)

            
        val_loss = session.run(cost, feed_dict=feed_dict_val)
        epoch = i    
        show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
#             saver.save(session, 'colorization-model') 


train(num_iteration=5)


Complete reading input data. Will Now print a snippet of it
Number of files in Training-set:		5000
Number of files in Validation(test)-set:	1000
Training Epoch 1 --- Training Accuracy: 100.0%, Validation(test) Accuracy: 100.0%,  Validation Loss: 0.000
Training Epoch 2 --- Training Accuracy: 100.0%, Validation(test) Accuracy: 100.0%,  Validation Loss: 0.000
Training Epoch 3 --- Training Accuracy: 100.0%, Validation(test) Accuracy: 100.0%,  Validation Loss: 0.000
Training Epoch 4 --- Training Accuracy: 100.0%, Validation(test) Accuracy: 100.0%,  Validation Loss: 0.000
Training Epoch 5 --- Training Accuracy: 100.0%, Validation(test) Accuracy: 100.0%,  Validation Loss: 0.000


In [141]:
import matplotlib.pyplot as plt

pic = np.array([trans[i] for i in kmeans.labels_])



In [ ]:
pic= pic.reshape(6000,32,32,3)
plt.imshow(pic[5000])
print pic[5000]